# Scraping

In [1]:
import pandas as pd
import requests
page = requests.get("https://www.almasirah.news/category/8/100/")
page

<Response [200]>

In [2]:
page.status_code


200

In [3]:
def scroll_page(driver, scroll_pause_time, screen_height): #will be used in the near future
    i = 1
    while i <= 250:
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
        i+=1
        time.sleep(scroll_pause_time)
        scroll_height= driver.execute_script("return document.body.scrollHeight;")
        if (screen_height) * i > scroll_height:
            break


In [4]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
driver = webdriver.Chrome(r"C:\Users\uriel\Downloads\chromedriver_win32\chromedriver.exe")
driver.get(fr"https://www.almasirah.news/category/8/100/")

C:\Users\uriel\AppData\Local\Temp/ipykernel_6836/1105863485.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\uriel\Downloads\chromedriver_win32\chromedriver.exe")


In [5]:
#Get links from website
from datetime import datetime
soup = bs(driver.page_source, 'html.parser')
all = soup.find_all(class_="card-header")
link_list=[]
date_list=[]
for i in all:
    for a in i.find_all('a', href=True):
        if(a.get('href').startswith('https://www.almasirah.news/post')):
            link_list.append(a.get('href'))

In [6]:
#Get text from each link
from tqdm import tqdm
text = []
for link in tqdm(link_list):
    text_list =[]
    driver.get(link)
    soup = bs(driver.page_source, 'html.parser')
    all_text = soup.find_all("p")
    for item in all_text:
        text_list.append(item.text)
    text.append(''.join(text_list))   

100%|██████████| 233/233 [18:57<00:00,  4.88s/it]


In [7]:
# Dates extraction 
import re 
dates=[]
for link in tqdm(link_list):
    driver.get(link)
    soup = bs(driver.page_source, 'html.parser')
    all_dates = soup.find_all(class_="btn btn-tag pull-left")
    try:
        match = re.search('\d{4}-\d{2}-\d{2}', str(all_dates))
        dates.append(match.group())
    except AttributeError:
        dates.append('No date')

100%|██████████| 233/233 [19:51<00:00,  5.11s/it]


In [8]:
dates 

['No date',
 'No date',
 'No date',
 '2022-02-03',
 '2021-12-18',
 '2021-12-02',
 '2021-10-18',
 '2021-10-07',
 '2021-09-20',
 '2021-09-02',
 '2021-08-19',
 '2021-08-09',
 'No date',
 '2021-06-03',
 'No date',
 '2021-05-12',
 '2021-05-11',
 '2021-05-10',
 '2021-05-09',
 '2021-05-08',
 '2021-05-07',
 '2021-05-07',
 '2021-05-05',
 '2021-05-05',
 '2021-05-04',
 '2021-05-03',
 '2021-05-02',
 '2021-05-01',
 '2021-04-30',
 '2021-04-29',
 'No date',
 'No date',
 'No date',
 'No date',
 'No date',
 'No date',
 'No date',
 'No date',
 '2021-04-20',
 '2021-04-20',
 '2021-04-18',
 '2021-04-17',
 '2021-04-16',
 '2021-04-15',
 '2021-04-14',
 '2021-04-14',
 '2021-04-09',
 'No date',
 '2021-03-10',
 '2021-02-19',
 '2021-01-03',
 '2020-12-09',
 '2020-10-29',
 '2020-10-19',
 'No date',
 '2020-09-14',
 '2020-08-30',
 '2020-08-20',
 '2020-08-08',
 '2020-06-18',
 'No date',
 'No date',
 'No date',
 '2020-05-20',
 '2020-05-20',
 '2020-05-19',
 '2020-05-18',
 '2020-05-17',
 '2020-05-16',
 '2020-05-15',
 '20

In [9]:
main_dict = {'Text':text, 'Date': dates,'Link':link_list}
main_df = pd.DataFrame(main_dict)
main_df.head()

,Text,Date,Link
0,شبكة المسيرة الإعلامية، شبكة يمنية متنوعة تسعى...,No date,https://www.almasirah.news/post/209679/%D9%83%...
1,شبكة المسيرة الإعلامية، شبكة يمنية متنوعة تسعى...,No date,https://www.almasirah.news/post/209384/%28%D9%...
2,شبكة المسيرة الإعلامية، شبكة يمنية متنوعة تسعى...,No date,https://www.almasirah.news/post/207375/%28%D9%...
3,شبكة المسيرة الإعلامية، شبكة يمنية متنوعة تسعى...,2022-02-03,https://www.almasirah.news/post/205515/%28%D9%...
4,شبكة المسيرة الإعلامية، شبكة يمنية متنوعة تسعى...,2021-12-18,https://www.almasirah.news/post/202121/%28%D9%...


# Find UAE in text #

In [10]:
import re
from tqdm import tqdm
uae_temp_list=[]
uae_list=[]
for i in tqdm(range(len(main_df))):
    text = main_df['Text'][i]
    date = main_df['Date'][i]
    matches=["الأنظمة العميلة", "النظام السعودي", "النظام الإماراتي", "العدوان السعودي الأمريكي", "المنافقون", "المرتزقة والدواعش الإماراتية", "ادوات"]
    for word in matches:
        if len(re.findall(word,text)) != 0:
            findall_uae_list =re.findall(r'[^.]*' + word + r'[^.]*\.', text)
            for find in findall_uae_list:
                uae_temp_list.append([find,date])
uae_df = pd.DataFrame(uae_temp_list, columns=['UAE_Text', 'Date'])

        

100%|██████████| 233/233 [00:16<00:00, 14.43it/s]


In [11]:
# Change to datetime
for i in range(len(uae_df)):
    if uae_df['Date'][i] != 'No date':
        uae_df['Date'][i] = re.sub("[^0-9]","",uae_df['Date'][i])
        uae_df['Date'][i] = pd.to_datetime(uae_df['Date'][i])
        uae_df['Date'][i] = uae_df['Date'][i].date()

In [12]:
#simple search for text specific dates
from_21_and_over_df= uae_df#[0:230]
from_21_and_over_df = from_21_and_over_df.reindex(index=from_21_and_over_df.index[::-1]).reset_index(drop="True")
from_21_and_over_df.tail()

,UAE_Text,Date
415,هذه الاستراتيجية فعلاً يبنون عليها سياساتهم وت...,No date
416,قال عنهم: {وَإِذَا خَلَوْا عَضُّوا عَلَيْكُمُ ...,No date
417,فطغيان العناوين والسياسات التي تريدها أمريكا م...,No date
418,ثم على المستوى العملي، احتلال بلدان، قتل لمئات...,No date
419,وفي نفس الوقت كلما حصلت تطورات، أو أحداث، يصطف...,No date


# Sentiment Analysis

In [13]:
import torch

device = torch.device("cpu")

In [14]:
from transformers import pipeline
sentiment_list=[]
sentiment_text=[]
sentiment_date=[]
for i in tqdm(range(len(from_21_and_over_df))):
    sa_text = from_21_and_over_df['UAE_Text'][i]
    if len(sa_text) > 512:
            #sa_text = sa_text.split()
            sa_text = sa_text[0:512]
            #sa_text = ''.join(sa_text)
    sa = pipeline('text-classification', model='CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment')
    sentiment_list.append(sa(sa_text))
    sentiment_text.append(sa_text)
    sentiment_date.append(from_21_and_over_df['Date'][i])

100%|██████████| 420/420 [1:57:54<00:00, 16.84s/it]


In [15]:
final_dict={"UAE_Text": sentiment_text, "Date": sentiment_date, "Sentiment":sentiment_list}
final_df_with_sentiment = pd.DataFrame(final_dict) 
final_df_with_sentiment.head()

,UAE_Text,Date,Sentiment
0,ثبت أن قوة الاستكبار اليوم وعلى رأسها أمريكا ...,2016-12-11,"[{'label': 'negative', 'score': 0.996241331100..."
1,\nلاحظوا مَا مِن دولة ولا من أمة إلا ولها جيش ...,No date,"[{'label': 'neutral', 'score': 0.6565144658088..."
2,\nالآن هناك مرحلة طرأت في الواقع الدولي، مجيء ...,No date,"[{'label': 'negative', 'score': 0.994984984397..."
3,\nتقييمنا لمجريات الأحداث خلال الفترة الماضية ...,No date,"[{'label': 'positive', 'score': 0.977029383182..."
4,رابعا: العناية بوحدة الصف الداخلي، هناك شغل كب...,2017-02-10,"[{'label': 'positive', 'score': 0.771841645240..."


In [16]:
final_df_with_sentiment.to_excel('uae sentiment_29032022.xlsx')

In [17]:
driver.quit

<bound method ChromiumDriver.quit of <selenium.webdriver.chrome.webdriver.WebDriver (session="f90e3bf21083df9009d819d9f094f2a6")>>

In [18]:
final_df_with_sentiment['UAE_Text'].iloc[0]

'\xa0ثبت أن قوة الاستكبار اليوم وعلى رأسها أمريكا وإسرائيل تفاقم مشاكل البشرية وتفسد في الأرض وتعتدي على الشعوب وتنهب الخيرات وتصنع الحروب والأزمات ولا تقدم للبشرية إلا المزيد من المآسي والنكبات، وزاد من سوء الأمر في عالمنا الإسلامي خصوصا في المنطقة العربية التبعية العمياء من بعض الدول التي تقدم نفسها على أنها تمثل الإسلام كما هو حال النظام السعودي المنافق، الذي جعل من نفسه أداة الشر لتنفيذ مؤامرات الأعداء وهدم كيان الأمة من الداخل، وهو بلا شك امتداد ظلامي ظالم لقوى الاستكبار، ويمثل حالة الانحراف والتحريف داخل '